In [1]:
!pip install ultralytics gradio opencv-python-headless pillow fpdf2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 18.4 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()

Saving best.pt to best.pt


In [32]:
from ultralytics import YOLO
import gradio as gr
import cv2
import numpy as np
from PIL import Image
from datetime import datetime
import matplotlib.pyplot as plt
import io
from fpdf import FPDF

# Model yükle
model = YOLO('/content/best.pt')
print(f"✅ Model: {model.names}")

# Rapor geçmişi
report_history = []

def analyze(image, conf=0.1):
    if image is None:
        return None, " Resim yükleyin!", None

    img = np.array(image)
    results = model.predict(cv2.cvtColor(img, cv2.COLOR_RGB2BGR), conf=conf, verbose=False)
    annotated = cv2.cvtColor(results[0].plot(line_width=3), cv2.COLOR_BGR2RGB)
    detections = len(results[0].boxes)

    if detections == 0:
        return Image.fromarray(annotated), "Hasar bulunamadı.", None

    damages = []
    total = 0
    for i, box in enumerate(results[0].boxes):
        c = float(box.conf[0])
        cost = int(5000 + 15000 * c)
        total += cost
        damages.append({'no': i+1, 'conf': c, 'cost': cost})

    risk = "YÜKSEK" if total > 50000 else "ORTA" if total > 25000 else "DÜŞÜK"

    report = {
        'date': datetime.now().strftime('%d.%m.%Y %H:%M'),
        'id': f"RPT-{len(report_history)+1:03d}",
        'count': detections,
        'damages': damages,
        'total': total,
        'risk': risk
    }
    report_history.append(report)

    summary = f"{detections} hasar tespit edildi | Toplam Maliyet: ₺{total:,} | Risk Seviyesi: {risk}"

    chart = create_chart(damages) if detections > 1 else None

    return Image.fromarray(annotated), summary, chart


def create_chart(damages):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), facecolor='#1a1a1a')
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD']
    labels = [f"Hasar {d['no']}" for d in damages]
    costs = [d['cost'] for d in damages]
    confs = [d['conf']*100 for d in damages]

    ax1.bar(labels, costs, color=colors[:len(damages)], edgecolor='white', linewidth=1.5)
    ax1.set_title('Tahmini Maliyet (₺)', fontweight='bold', color='white', fontsize=14)
    ax1.set_ylabel('Tutar (₺)', color='white', fontweight='bold')
    ax1.tick_params(colors='white')
    ax1.set_facecolor('#2a2a2a')
    ax1.spines['bottom'].set_color('white')
    ax1.spines['left'].set_color('white')
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.grid(axis='y', alpha=0.3, linestyle='--', color='gray')

    ax2.pie(confs, labels=labels, colors=colors[:len(damages)], autopct='%1.0f%%',
            textprops={'color': 'white', 'fontweight': 'bold'},
            wedgeprops={'edgecolor': 'white', 'linewidth': 2})
    ax2.set_title('Güvenilirlik Dağılımı', fontweight='bold', color='white', fontsize=14)
    ax2.set_facecolor('#2a2a2a')

    plt.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=120, bbox_inches='tight', facecolor='#1a1a1a')
    buf.seek(0)
    plt.close()
    return Image.open(buf)


def show_reports():
    if not report_history:
        return "Henüz rapor yok."

    text = "════════════════════════════════════════════════════\n"
    text += "            HASAR ANALİZ RAPOR GEÇMİŞİ            \n"
    text += "════════════════════════════════════════════════════\n"

    for r in reversed(report_history):
        text += f"\n── {r['id']} ─────────────────────────────────────────\n"
        text += f"  Tarih: {r['date']}\n"
        text += f"  Tespit: {r['count']} adet hasar\n"
        text += f"  Toplam: ₺{r['total']:,}\n"
        text += f"  Risk: {r['risk']}\n"
        text += "────────────────────────────────────────────────────\n"
        for d in r['damages']:
            text += f"    • Hasar {d['no']}: %{d['conf']*100:.1f} güven → ₺{d['cost']:,}\n"
        text += "────────────────────────────────────────────────────\n"

    return text


def download_pdf():
    """Tüm raporları PDF olarak indir"""
    if not report_history:
        return None

    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)

        pdf.set_font("Arial", "B", 18)
        pdf.cell(0, 15, "ARAC HASAR ANALIZ RAPORU", ln=True, align="C")
        pdf.ln(3)

        pdf.set_font("Arial", "", 11)
        pdf.cell(0, 10, f"Rapor Tarihi: {datetime.now().strftime('%d.%m.%Y %H:%M')}", ln=True, align="C")
        pdf.ln(8)

        for idx, r in enumerate(reversed(report_history)):
            if pdf.get_y() > 250:
                pdf.add_page()

            pdf.set_font("Arial", "B", 14)
            pdf.set_fill_color(220, 220, 220)
            pdf.cell(0, 10, f"Rapor No: {r['id']}", ln=True, fill=True)
            pdf.ln(2)

            pdf.set_font("Arial", "", 11)
            pdf.cell(55, 8, "Tarih:", 0)
            pdf.cell(0, 8, r['date'], ln=True)

            pdf.cell(55, 8, "Tespit Edilen Hasar:", 0)
            pdf.cell(0, 8, f"{r['count']} adet", ln=True)

            pdf.cell(55, 8, "Toplam Maliyet:", 0)
            pdf.cell(0, 8, f"TL {r['total']:,}", ln=True)

            pdf.cell(55, 8, "Risk Seviyesi:", 0)
            pdf.cell(0, 8, r['risk'], ln=True)
            pdf.ln(3)

            pdf.set_font("Arial", "B", 11)
            pdf.cell(0, 8, "Hasar Detaylari:", ln=True)
            pdf.set_font("Arial", "", 10)

            for d in r['damages']:
                if pdf.get_y() > 270:
                    pdf.add_page()
                pdf.cell(15, 7, "", 0)
                pdf.cell(0, 7, f"Hasar {d['no']}: %{d['conf']*100:.1f} guvenilirlik - TL {d['cost']:,}", ln=True)

            pdf.ln(5)

            if idx < len(report_history) - 1:
                if pdf.get_y() > 270:
                    pdf.add_page()
                else:
                    pdf.set_draw_color(150, 150, 150)
                    pdf.line(10, pdf.get_y(), 200, pdf.get_y())
                    pdf.ln(8)

        pdf_filename = f"hasar_raporu_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
        pdf_path = f"/content/{pdf_filename}"
        pdf.output(pdf_path)

        print(f"✅ PDF basariyla olusturuldu: {pdf_path}")
        return pdf_path

    except Exception as e:
        print(f"❌ PDF olusturma hatasi: {str(e)}")
        import traceback
        traceback.print_exc()
        return None


# CSS - Gradio footer'ı gizle
custom_css = """
#main-title {
    text-align: center;
    background: linear-gradient(135deg, #2c3e50 0%, #34495e 100%);
    padding: 30px;
    border-radius: 15px;
    margin-bottom: 20px;
    box-shadow: 0 10px 30px rgba(0,0,0,0.5);
    border: 2px solid #7f8c8d;
}

#main-title h1 {
    color: #ecf0f1;
    font-size: 2.5em;
    margin: 0;
    text-shadow: 2px 2px 4px rgba(0,0,0,0.5);
    letter-spacing: 2px;
}

#main-title p {
    color: #bdc3c7;
    font-size: 1.1em;
    margin: 10px 0 0 0;
}

.gradio-container {
    background: linear-gradient(135deg, #1a1a1a 0%, #2c2c2c 100%);
}

#analyze-btn {
    background: linear-gradient(135deg, #7f8c8d 0%, #95a5a6 100%) !important;
    font-size: 1.3em !important;
    font-weight: bold !important;
    padding: 15px !important;
    border: 2px solid #bdc3c7 !important;
    box-shadow: 0 5px 15px rgba(127,140,141,0.4) !important;
    transition: all 0.3s ease !important;
    color: #1a1a1a !important;
}

#analyze-btn:hover {
    transform: translateY(-2px);
    box-shadow: 0 8px 20px rgba(127,140,141,0.6) !important;
    background: linear-gradient(135deg, #95a5a6 0%, #bdc3c7 100%) !important;
}

#report-btn {
    background: linear-gradient(135deg, #34495e 0%, #2c3e50 100%) !important;
    font-size: 1.1em !important;
    font-weight: bold !important;
    border: 2px solid #7f8c8d !important;
}

#pdf-btn {
    background: linear-gradient(135deg, #e74c3c 0%, #c0392b 100%) !important;
    font-size: 1.1em !important;
    font-weight: bold !important;
    border: 2px solid #e74c3c !important;
}

.tab-nav button {
    font-size: 1.1em !important;
    font-weight: bold !important;
    padding: 12px 24px !important;
}

/* GRADIO FOOTER GİZLE */
footer {
    display: none !important;
}

.gradio-container > footer {
    display: none !important;
}
"""

with gr.Blocks(theme=gr.themes.Soft(primary_hue="slate", secondary_hue="gray"), css=custom_css, title="Araç Hasar Analizi") as demo:

    gr.HTML("""
        <div id="main-title">
            <h1>ARAÇ HASAR ANALİZ SİSTEMİ</h1>
            <p>Yapay Zeka Destekli Otomatik Hasar Tespiti ve Maliyet Analizi</p>
        </div>
    """)

    with gr.Tabs():
        with gr.Tab("Hasar Analizi", elem_id="analysis-tab"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### 1. Araç Resmini Yükleyin")
                    img_in = gr.Image(type="pil", label="", height=350)

                    gr.Markdown("### 2. Tespit Hassasiyetini Ayarlayın")
                    conf = gr.Slider(
                        0.01, 0.5, 0.1,
                        label="Güvenilirlik Eşiği",
                        info="Değeri düşürürseniz daha fazla hasar tespit edilir"
                    )

                    btn = gr.Button("ANALİZ ET", variant="primary", size="lg", elem_id="analyze-btn")

                with gr.Column(scale=1):
                    gr.Markdown("### 3. Tespit Sonuçları")
                    img_out = gr.Image(label="", height=350)

                    gr.Markdown("### Özet Bilgi")
                    summary = gr.Textbox(
                        label="",
                        lines=2,
                        show_copy_button=True,
                        container=True
                    )

            gr.Markdown("---")
            gr.Markdown("### Detaylı İstatistikler")
            chart = gr.Image(label="", height=300, visible=True)

            btn.click(analyze, [img_in, conf], [img_out, summary, chart])

        with gr.Tab("Rapor Geçmişi", elem_id="reports-tab"):
            gr.Markdown("""
                ### Tüm Analiz Kayıtları
                Yapılan tüm hasar analizlerinin detaylı kayıtlarını görüntüleyin ve PDF olarak indirin.
            """)

            with gr.Row():
                report_btn = gr.Button("Raporları Yenile", variant="primary", size="lg", elem_id="report-btn", scale=1)
                pdf_btn = gr.Button("PDF Olarak İndir", variant="secondary", size="lg", elem_id="pdf-btn", scale=1)

            report_text = gr.Textbox(
                label="",
                lines=30,
                show_copy_button=True,
                container=True
            )

            pdf_file = gr.File(label="İndirilen PDF", visible=True)

            report_btn.click(show_reports, [], report_text)
            pdf_btn.click(download_pdf, [], pdf_file)

# FOOTER GİZLEME İÇİN show_api=False
demo.launch(share=True, debug=True, show_api=False)

✅ Model: {0: 'damage'}


/tmp/ipython-input-3883412118.py:263: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(primary_hue="slate", secondary_hue="gray"), css=custom_css, title="Araç Hasar Analizi") as demo:
/tmp/ipython-input-3883412118.py:263: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(primary_hue="slate", secondary_hue="gray"), css=custom_css, title="Araç Hasar Analizi") as demo:
/tmp/ipython-input-3883412118.py:329: DeprecationWarning: The 'show_api' parameter in launch() will be removed in Gradio 6.0. You will need to use the 'footer_links' parameter instead. To replicate show_api=False, In Gradio 6.0, use footer_links=['gradio', 'settings'].
  demo.launch(share=True, debug=True, show_api=False)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://523ae1be36a65c4da8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

✅ PDF basariyla olusturuldu: /content/hasar_raporu_20251220_111547.pdf
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://523ae1be36a65c4da8.gradio.live


In [12]:
print(model.names)

{0: 'damage'}
